In [1]:
import numpy as np
import pandas as pd
from epiweeks import Week
import matplotlib.pyplot as plt
from mosqlient import get_predictions
from mosqlient.models.score import Scorer

import seaborn as sns
import dataframe_image as dfi
from itertools import product 

In [2]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('data/dengue_uf.csv.gz')

data.head()

,uf,date,casos
0,AC,2010-01-03,869
1,AC,2010-01-10,1003
2,AC,2010-01-17,1010
3,AC,2010-01-24,1397
4,AC,2010-01-31,2000


In [4]:
def get_cases(data, state):
    data_ = data.loc[data.uf == state]
    
    data_.loc[:, 'date']  = pd.to_datetime(data_.date)

    data_ = data_.rename(columns = {'date':'dates'})
    
    return data_

Teams and model_id na plataforma: 


| Team                     | Model id |
| ------------------------ | ------------- |
| D-fense                  | ------------  |
| Dobby Data               | 21            |
| GeoHealth                | 25,26         |
| Global Health Resilience | 22            |
| BB-M                     | 30            |
| Ki-Dengu Peppa           | 27, 28        |
| DS_OKSTATE               | 29            |
 


The cell below fetch the predictions from the API and compute the scores for the 52 weeks interval saved on the `df_score_end_52` dataframe and for the 26 weeks interval saved on the `df_score_end_26` dataframe. 

In [5]:
def get_preds(model_id):

    list_of_preds = []
    
    if model_id == 27:

        for p_id in np.arange(250,260):

            print(p_id)
            
            list_of_preds.append(get_predictions(id = int(p_id)))

    elif model_id == 28:

        for p_id in np.arange(260,270):
            
            list_of_preds.append(get_predictions(id = int(p_id)))

    else: 
        list_of_preds = get_predictions(model_id = model_id)


    return list_of_preds 


In [6]:
%%time 
df_score_end_52  = pd.DataFrame()
df_score_end_26  = pd.DataFrame()

for model_id in [21, 22, 25,26, 27, 28, 29, 30]:

    list_of_preds = get_preds(model_id = model_id)

    for idx in np.arange(0, len(list_of_preds)): 

        df = list_of_preds[idx].to_dataframe()
        state = df.adm_1.unique()[0]

        if state in ['AM', 'CE', 'GO', 'PR', 'MG']:
            preds_id = list_of_preds[idx].id
            df_true = get_cases(data, state)
    
            score = Scorer(df_true = df_true, ids = [preds_id], confidence_level=0.9)

            year = pd.to_datetime(df.dates).dt.year.max()

            score.set_date_range(start_date = Week(year-1, 41).startdate().strftime('%Y-%m-%d'),
                                 end_date= Week(year, 40).startdate().strftime('%Y-%m-%d'))
            

            year = pd.to_datetime(df.dates).dt.year.max()
            df_score_52 = score.summary
            df_score_52['state'] = state
            df_score_52['year'] = year
            if model_id == 26:
                df_score_52['model_id'] = 25
            else:
                df_score_52['model_id'] = model_id

            score.set_date_range(start_date = Week(year-1, 41).startdate().strftime('%Y-%m-%d'),
                                 end_date= Week(year, 14).startdate().strftime('%Y-%m-%d'))

            df_score_26 = score.summary
            df_score_26['state'] = state
            df_score_26['year'] = year
            
            if model_id == 26:
                df_score_26['model_id'] = 25
            else:
                df_score_26['model_id'] = model_id

            df_score_end_52 = pd.concat([df_score_end_52, df_score_52])
            df_score_end_26 = pd.concat([df_score_end_26, df_score_26])
            

250
251
252
253
254
255
256
257
258
259
CPU times: user 4.25 s, sys: 304 ms, total: 4.55 s
Wall time: 2min 13s


Checking the shape od the data:

In [7]:
df_score_end_52.shape

(70, 8)

In [8]:
df_score_end_26.shape

(70, 8)

Checking for null values:

In [9]:
df_score_end_52.isnull().sum()

mae               0
mse               0
crps              0
log_score         0
interval_score    0
state             0
year              0
model_id          0
dtype: int64

In [10]:
df_score_end_26.isnull().sum()

mae               0
mse               0
crps              0
log_score         0
interval_score    0
state             0
year              0
model_id          0
dtype: int64

In [11]:
df_score_end_52.model_id.value_counts()

model_id
21    10
22    10
25    10
27    10
28    10
29    10
30    10
Name: count, dtype: int64

In [12]:
df_score_end_26.model_id.value_counts()

model_id
21    10
22    10
25    10
27    10
28    10
29    10
30    10
Name: count, dtype: int64

## Analysing the scores

Functions to rank the models and gen the figures:

In [13]:
def compute_the_rank_by_scores(df_52_, df_26_): 
    '''
    Rank the models by each of the scores S1,..., S6 proposed. 
    '''
    
    S1 = df_52_[['crps', 'model_id']].copy()
    S1['rank'] = df_52_['crps'].rank(method='dense', ascending=True)
    S1['score'] = 'S1'

    S2 = df_26_[['crps', 'model_id']].copy()
    S2['rank'] = df_26_['crps'].rank(method='dense', ascending=True)
    S2['score'] = 'S2'

    S3 = df_52_[['log_score', 'model_id']].copy()
    S3['rank'] = df_52_['log_score'].rank(method='dense', ascending=False)
    S3['score'] = 'S3'

    S4 = df_26_[['log_score', 'model_id']].copy()
    S4['rank'] = df_26_['log_score'].rank(method='dense', ascending=False)
    S4['score'] = 'S4'

    S5 = df_52_[['interval_score', 'model_id']].copy()
    S5['rank'] = df_52_['interval_score'].rank(method='dense', ascending=True)
    S5['score'] = 'S5'

    S6 = df_26_[['interval_score', 'model_id']].copy()
    S6['rank'] = df_26_['interval_score'].rank(method='dense', ascending=True)
    S6['score'] = 'S6'

    S = pd.concat([S1, S2, S3, S4, S5, S6]).reset_index(drop = True)

    S = pd.pivot_table(S, values = 'rank', index='model_id',
                      columns = ['score'])

    S.columns = 'rank_' + S.columns 

    return S


def compute_the_rank_sep(df_, scores = ['S1', 'S2', 'S3']): 
    '''
    Rank the models by each of the scores proposed. 
    '''
    
    S1 = df_[['crps', 'model_id']].copy()
    S1['rank'] = df_['crps'].rank(method='dense', ascending=True)
    S1['score'] = scores[0]

    S3 = df_[['log_score', 'model_id']].copy()
    S3['rank'] = df_['log_score'].rank(method='dense', ascending=False)
    S3['score'] = scores[1]

    S5 = df_[['interval_score', 'model_id']].copy()
    S5['rank'] = df_['interval_score'].rank(method='dense', ascending=True)
    S5['score'] = scores[2]

    S = pd.concat([S1, S3, S5]).reset_index(drop = True)

    S = pd.pivot_table(S, values = 'rank', index='model_id',
                      columns = ['score'])

    S.columns = 'rank_' + S.columns 

    return S

def compute_the_final_rank_sep(S, scores = ['rank_S1', 'rank_S3', 'rank_S5']):
    '''
    Get the final rank of the models based on the output of the compute_the_rank_by_scores function. 
    '''
    v_r_end = []
    for model_id in S.index: 
    
        rank_final = ((1/S.loc[model_id][scores[0]]) +
                     (1/S.loc[model_id][scores[1]]) +
                     (1/S.loc[model_id][scores[2]]))
    
        v_r_end.append(rank_final)
    
    df_rank = pd.DataFrame()
    
    df_rank['model_id'] = S.index
    df_rank['rank_values'] = v_r_end

    df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
    df_rank['state'] = state
    df_rank['year'] = year

    df_rank = df_rank.set_index('model_id')

    df_rank = df_rank.rename(columns = {'rank':'composite_rank'})

    df_rank = S.merge(df_rank[['composite_rank', 'state', 'year']], left_index = True, right_index = True)

    for c in [scores[0], scores[1], scores[2], 'composite_rank']:

        df_rank[c] = df_rank[c].astype(int)

    return df_rank.sort_values(by ='composite_rank')
    
    
def compute_the_final_rank(S):
    '''
    Get the final rank of the models based on the output of the compute_the_rank_by_scores function. 
    '''
    v_r_end = []
    for model_id in S.index: 
    
        rank_final = ((1/S.loc[model_id]['rank_S1']) +
                     (1/S.loc[model_id]['rank_S2']) +
                     (1/S.loc[model_id]['rank_S3']) + 
                     (1/S.loc[model_id]['rank_S4']) + 
                     (1/S.loc[model_id]['rank_S5']) + 
                     (1/S.loc[model_id]['rank_S6']))
    
        v_r_end.append(rank_final)
    
    df_rank = pd.DataFrame()
    
    df_rank['model_id'] = S.index
    df_rank['rank_values'] = v_r_end

    df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
    df_rank['state'] = state
    df_rank['year'] = year

    df_rank = df_rank.set_index('model_id')

    df_rank = df_rank.rename(columns = {'rank':'composite_rank'})

    df_rank = S.merge(df_rank[['composite_rank', 'state', 'year']], left_index = True, right_index = True)

    for c in ['rank_S1', 'rank_S2', 'rank_S3', 
                                         'rank_S4', 'rank_S5', 'rank_S6', 'composite_rank']:

        df_rank[c] = df_rank[c].astype(int)

    return df_rank.sort_values(by ='composite_rank')

def color_and_save_table(df_rank, filename, subset = ['rank_S1', 'rank_S2', 'rank_S3', 
                                             'rank_S4', 'rank_S5', 'rank_S6', 'composite_rank']):

    cm = sns.color_palette('light:b_r', as_cmap=True)

    rename_models = {30: 'BB-M', 21: 'Dobby data', 25: 'GeoHealth', 
                22: 'Global Health Resilience', 29: 'DS_OKSTATE',
                27: 'Ki-Dengu Peppa model 1', 
                28: 'Ki-Dengu Peppa model 2'}

    df_rank = df_rank.rename(index=rename_models)

    df_rank.index.name = 'Model'

    df_rank = df_rank.style.background_gradient(subset=subset, cmap=cm)

    dfi.export(df_rank, f'./figures/{filename}.png')

    return df_rank



Get the final rank for each state and year and create a figure of the table colored according to the rank value: 

In [14]:
df_52 = df_score_end_52
df_26 = df_score_end_26

In [15]:
%%time 
for state, year in product(['AM','CE','GO','PR','MG'], [2023, 2024]):
    df_52_ = df_52.loc[(df_52.state == state) & (df_52.year == year)]
    df_26_ = df_26.loc[(df_26.state == state) & (df_26.year == year)]

    S_52 = compute_the_rank_sep(df_52_, scores = ['S1', 'S3', 'S5'])

    R_52 = compute_the_final_rank_sep(S_52, scores = ['rank_S1', 'rank_S3', 'rank_S5'])

    R_52.to_csv(f'tables/ranking_52_{state}_{year}.csv')

    S_26 = compute_the_rank_sep(df_26_, scores = ['S2', 'S4', 'S6'])

    R_26 = compute_the_final_rank_sep(S_26, scores = ['rank_S2', 'rank_S4', 'rank_S6'])

    R_26.to_csv(f'tables/ranking_26_{state}_{year}.csv')


CPU times: user 108 ms, sys: 12.1 ms, total: 120 ms
Wall time: 174 ms


In [16]:
%%time 
for state, year in product(['AM','CE','GO','PR','MG'], [2023, 2024]):
    df_52_ = df_52.loc[(df_52.state == state) & (df_52.year == year)]
    df_26_ = df_26.loc[(df_26.state == state) & (df_26.year == year)]

    S_end = compute_the_rank_by_scores(df_52_, df_26_)

    R_end = compute_the_final_rank(S_end)
    R_end.to_csv(f'tables/ranking_{state}_{year}.csv')

    
    color_and_save_table(R_end, filename = f'ranking_{state}_{year}',
                         subset = ['rank_S1', 'rank_S2', 'rank_S3', 
                                             'rank_S4', 'rank_S5', 'rank_S6', 'composite_rank'])
    


CPU times: user 560 ms, sys: 157 ms, total: 717 ms
Wall time: 5.08 s


Get the final rank for each state based on the rank of each year:

In [17]:
for state in ['AM', 'CE', 'GO', 'PR', 'MG']:
    df_r_2023 = pd.read_csv(f'tables/ranking_{state}_2023.csv', index_col = 'model_id')
    
    df_r_2024 = pd.read_csv(f'tables/ranking_{state}_2024.csv', index_col = 'model_id')
    
    v_r_end = []
    for model_id in df_r_2023.index: 
        
            rank_final = ((1/df_r_2023.loc[model_id]['composite_rank']) +
                         (1/df_r_2024.loc[model_id]['composite_rank']))
        
            v_r_end.append(rank_final)
        
    df_rank = pd.DataFrame()
        
    df_rank['model_id'] = df_r_2023.index
    df_rank['rank_values'] = v_r_end
    
    df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
    df_rank['state'] = state
   
    df_rank.to_csv(f'tables/ranking_{state}.csv', index = False)

In [18]:
for state in ['AM', 'CE', 'GO', 'PR', 'MG']:
    for w in ['26', '52']:
    
        df_r_2023 = pd.read_csv(f'tables/ranking_{w}_{state}_2023.csv', index_col = 'model_id')
        
        df_r_2024 = pd.read_csv(f'tables/ranking_{w}_{state}_2024.csv', index_col = 'model_id')
        
        v_r_end = []
        for model_id in df_r_2023.index: 
            
                rank_final = ((1/df_r_2023.loc[model_id]['composite_rank']) +
                             (1/df_r_2024.loc[model_id]['composite_rank']))
            
                v_r_end.append(rank_final)
            
        df_rank = pd.DataFrame()
            
        df_rank['model_id'] = df_r_2023.index
        df_rank['rank_values'] = v_r_end
        
        df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
        df_rank['state'] = state
       
        df_rank.to_csv(f'tables/ranking_{w}_{state}.csv', index = False)

Get the global rank based on the rank of each mandatory state: 

In [19]:
state = 'PR'
pd.read_csv(f'tables/ranking_{state}_2023.csv')

,model_id,rank_S1,rank_S2,rank_S3,rank_S4,rank_S5,rank_S6,composite_rank,state,year
0,30,3,1,1,1,2,1,1,PR,2023
1,27,2,4,5,6,1,2,2,PR,2023
2,21,1,2,7,5,6,4,3,PR,2023
3,25,4,5,3,3,3,3,4,PR,2023
4,22,6,6,2,2,4,6,5,PR,2023
5,29,5,3,4,4,7,5,6,PR,2023
6,28,7,7,6,7,5,7,7,PR,2023


In [20]:
pd.read_csv(f'tables/ranking_52_{state}_2023.csv')

,model_id,rank_S1,rank_S3,rank_S5,composite_rank,state,year
0,30,3,1,2,1,PR,2023
1,27,2,5,1,2,PR,2023
2,21,1,7,6,3,PR,2023
3,22,6,2,4,4,PR,2023
4,25,4,3,3,5,PR,2023
5,29,5,4,7,6,PR,2023
6,28,7,6,5,7,PR,2023


In [21]:
pd.read_csv(f'tables/ranking_26_{state}_2023.csv')

,model_id,rank_S2,rank_S4,rank_S6,composite_rank,state,year
0,30,1,1,1,1,PR,2023
1,21,2,5,4,2,PR,2023
2,27,4,6,2,3,PR,2023
3,25,5,3,3,4,PR,2023
4,22,6,2,6,5,PR,2023
5,29,3,4,5,6,PR,2023
6,28,7,7,7,7,PR,2023


In [22]:
def get_global_rank(year=None, weeks =None): 
    list_of_dfs = [] 

    if weeks !=None:

        label_weeks = f'{weeks}_'
    else: 
        label_weeks = '' 
    
    for state in ['AM', 'CE', 'GO', 'PR', 'MG']: 
        if year == None:
            col_name = 'rank'
            df_rank_ = pd.read_csv(f'tables/ranking_{state}.csv', index_col = 'model_id', usecols = ['model_id', col_name])
        else: 
            col_name = 'composite_rank'
            df_rank_ = pd.read_csv(f'tables/ranking_{label_weeks}{state}_{year}.csv', index_col = 'model_id', usecols = ['model_id', col_name])
            
        df_rank_ = df_rank_.rename(columns = {col_name: f'rank_{state}'} )
    
        list_of_dfs.append(df_rank_)
        
    df_rank_states = pd.concat(list_of_dfs,axis =1)

    v_r_end = []

    for model_id in df_rank_states.index: 
    
        rank_final = (
                             (1/df_rank_states.loc[model_id]['rank_AM']) +
                             (1/df_rank_states.loc[model_id]['rank_CE']) + 
                             (1/df_rank_states.loc[model_id]['rank_GO']) + 
                             (1/df_rank_states.loc[model_id]['rank_MG']) + 
                             (1/df_rank_states.loc[model_id]['rank_PR']))
        v_r_end.append(rank_final)
        
    df_rank = pd.DataFrame()
        
    df_rank['model_id'] = df_rank_states.index
    df_rank['rank_values'] = v_r_end
        
    df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
    
    df_rank.set_index('model_id',inplace = True)
    
    df_rank = df_rank.rename(columns = {'rank': 'global_rank'})
    
    df_rank = df_rank_states.merge(df_rank['global_rank'], left_index = True, right_index = True)
    
    df_rank = df_rank.sort_values(by = 'global_rank')
    
    df_rank = df_rank.astype(int)

    if year != None:
        df_rank['year'] = year

    return df_rank 
    
        
        

In [23]:
df_rank = get_global_rank(year=None)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_global',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank
Model,,,,,,
BB-M,1,1,2,1,2,1
Global Health Resilience,2,4,1,2,4,2
GeoHealth,2,3,4,3,1,3
DS_OKSTATE,1,5,7,7,7,4
Dobby data,3,2,5,5,3,5
Ki-Dengu Peppa model 1,4,7,3,4,6,6
Ki-Dengu Peppa model 2,5,6,6,6,5,7


In [24]:
year = 2023
df_rank = get_global_rank(year=year)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_global_{year}',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank,year
Model,,,,,,,
BB-M,1,2,1,1,1,1,2023
Dobby data,3,1,3,3,2,2,2023
Global Health Resilience,2,3,4,5,3,3,2023
Ki-Dengu Peppa model 1,7,6,2,2,4,4,2023
GeoHealth,4,4,7,4,5,5,2023
DS_OKSTATE,5,5,5,6,6,6,2023
Ki-Dengu Peppa model 2,6,7,6,7,7,7,2023


In [25]:
year = 2023
df_rank = get_global_rank(year=year, weeks = 52)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_52_global_{year}',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank,year
Model,,,,,,,
BB-M,1,2,1,1,1,1,2023
Dobby data,3,1,5,3,2,2,2023
Ki-Dengu Peppa model 1,7,5,2,2,3,3,2023
Global Health Resilience,2,3,3,4,4,4,2023
GeoHealth,4,4,7,5,5,5,2023
DS_OKSTATE,5,6,4,6,6,6,2023
Ki-Dengu Peppa model 2,6,7,6,7,7,7,2023


In [26]:
year = 2023
df_rank = get_global_rank(year=year, weeks = 26)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_26_global_{year}',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank,year
Model,,,,,,,
BB-M,1,2,1,1,1,1,2023
Dobby data,2,1,2,2,3,2,2023
Global Health Resilience,3,3,3,5,2,3,2023
Ki-Dengu Peppa model 1,7,6,4,3,4,4,2023
GeoHealth,4,4,7,4,5,5,2023
DS_OKSTATE,5,5,5,6,6,6,2023
Ki-Dengu Peppa model 2,6,7,6,7,7,7,2023


In [27]:
year = 2024
df_rank = get_global_rank(year=year)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_global_{year}',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank,year
Model,,,,,,,
GeoHealth,2,2,2,2,1,1,2024
Global Health Resilience,4,3,1,1,3,2,2024
BB-M,5,1,5,4,6,3,2024
DS_OKSTATE,1,6,7,7,7,4,2024
Ki-Dengu Peppa model 2,7,5,3,3,2,5,2024
Dobby data,6,4,4,6,4,6,2024
Ki-Dengu Peppa model 1,3,7,6,5,5,7,2024


In [28]:
year = 2024
df_rank = get_global_rank(year=year, weeks = 52)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_52_global_{year}',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank,year
Model,,,,,,,
Global Health Resilience,3,2,1,1,3,1,2024
GeoHealth,2,3,2,2,1,2,2024
BB-M,5,1,4,4,6,3,2024
DS_OKSTATE,1,6,7,7,7,4,2024
Dobby data,6,4,5,6,2,5,2024
Ki-Dengu Peppa model 2,7,5,3,3,4,6,2024
Ki-Dengu Peppa model 1,4,7,6,5,5,7,2024


In [29]:
year = 2024
df_rank = get_global_rank(year=year, weeks = 26)

df_rank_cor = color_and_save_table(df_rank, filename = f'ranking_26_global_{year}',
                         subset = ['rank_AM', 'rank_CE', 'rank_GO', 
                                             'rank_PR', 'rank_MG', 'global_rank'])

df_rank_cor

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,global_rank,year
Model,,,,,,,
GeoHealth,2,1,2,2,1,1,2024
Global Health Resilience,4,3,1,1,3,2,2024
DS_OKSTATE,1,5,7,7,7,3,2024
Ki-Dengu Peppa model 2,7,4,4,3,2,4,2024
BB-M,5,2,5,4,5,5,2024
Dobby data,6,4,3,6,4,6,2024
Ki-Dengu Peppa model 1,3,6,6,5,6,7,2024
